In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
from pathlib import Path

sys.path.insert(0, Path("../").resolve().as_posix())

In [ ]:
import tempfile
from component.scripts.gee import get_gee_vrt
from component.scripts.utils import get_pdf_path
import geopandas as gpd
import component.widget as cw
import io
import ee
from sepal_ui.scripts.utils import init_ee

In [ ]:
init_ee()

In [ ]:
json_file = io.StringIO(
    """{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"lat": 5.33469724544027, "lng": 13.0256336559457, "id": 1}, "geometry": {"type": "Point", "coordinates": [13.0256336559457, 5.33469724544027]}}, {"id": "1", "type": "Feature", "properties": {"lat": 5.31724397918854, "lng": 13.0145627442248, "id": 2}, "geometry": {"type": "Point", "coordinates": [13.0145627442248, 5.31724397918854]}}, {"id": "2", "type": "Feature", "properties": {"lat": 5.31816258449969, "lng": 13.0320916877829, "id": 3}, "geometry": {"type": "Point", "coordinates": [13.0320916877829, 5.31816258449969]}}, {"id": "3", "type": "Feature", "properties": {"lat": 5.48440733356101, "lng": 12.9075439309229, "id": 4}, "geometry": {"type": "Point", "coordinates": [12.9075439309229, 5.48440733356101]}}, {"id": "4", "type": "Feature", "properties": {"lat": 5.46236646346553, "lng": 12.9093890828764, "id": 5}, "geometry": {"type": "Point", "coordinates": [12.9093890828764, 5.46236646346553]}}]}"""
)
geometries = gpd.read_file(json_file)
alert = cw.CustomAlert()

In [ ]:
c_inputs = mosaics, image_size, sources, bands, square_size = (
    [2024, 2023],
    250,
    ["sentinel", "landsat"],
    "Red, Green, Blue",
    90,
)

In [ ]:
tmp_dir = Path(tempfile.mkdtemp())
input_file_path = tmp_dir / "test_points.csv"

pdf_filepath = get_pdf_path(
    input_file_path.stem, sources, bands, square_size, image_size
)

vrt_list, title_list = get_gee_vrt(
    geometries,
    mosaics,
    image_size,
    pdf_filepath.stem,
    bands,
    sources,
    alert,
    tmp_dir,
)

In [ ]:
alert

In [ ]:
from component.scripts.gee import get_ee_image
from component import parameter as cp


satellite_id = "sentinel_2"
year = 2024
satellites = cp.getSatellites(sources, year)
start = str(year) + "-01-01"
end = str(year) + "-12-31"
aoi = ee.Geometry.Polygon(
    [
        [
            [13.024513100356552, 5.333572819469696],
            [13.026757769061255, 5.333572819469696],
            [13.026757769061255, 5.335822103684232],
            [13.024513100356552, 5.335822103684232],
            [13.024513100356552, 5.333572819469696],
        ]
    ]
)
dataset, ee_image = get_ee_image(satellites, satellite_id, start, end, bands, aoi)
dataset.getInfo();

In [ ]:
satellites

In [ ]:
ee_image.select(0).reduceRegion(
    reducer=ee.Reducer.count(),
    geometry=aoi,
    scale=ee_image.projection().nominalScale(),
).values().get(0).getInfo()

In [ ]:
pixel_masked = (
    ee_image.select(0)
    .reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=aoi,
        scale=ee_image.projection().nominalScale(),
    )
    .get(band)
)

In [ ]:
# create the feature collection name
dataset = (
    ee.ImageCollection(satellites[satelliteId])
    .filterDate(start, end)
    .filterBounds(mask)
    .map(cp.getCloudMask(satelliteId))
)

clip = dataset.median().clip(mask).select(cp.getAvailableBands()[bands][satelliteId])

In [ ]:
from component.scripts.gee import down_buffer, getImage
from sepal_ui.scripts.utils import init_ee
from component.scripts.utils import min_diagonal

init_ee()
import ee

In [ ]:
geometry = geometries
size = image_size

In [ ]:
ee_pts = [ee.Geometry.Point(*g.centroid.coords) for g in geometry.geometry]

# get the optimal size buffer
size_list = [min_diagonal(g, size) for g in geometry.to_crs(3857).geometry]

# create the buffers
ee_buffers = [pt.buffer(s / 2).bounds() for pt, s in zip(ee_pts, size_list)]

In [ ]:
ee_buffers[0].getInfo()

In [ ]:
mask = ee_buffers[0]
year = mosaics[0]

getImage(sources, bands, aoi, year)

In [ ]:
getImage(sources, bands, mask, year)